In [1]:
import spacy

# Load the English model
nlp = spacy.load("en_core_web_sm")
print("spaCy and the English model are successfully installed.")


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import pandas as pd
import re

In [ ]:
import pandas as pd
import re

def extract_sections(text):
    # Initialize a dictionary to hold extracted information
    sections = {
        'Name': '',
        'Contact': '',
        'Professional Summary': '',
        'Skills': [],
        'Education': [],
        'Experience': [],
        'Certifications': [],
        'Accomplishments': [],
        'Achievements': [],
        'Hobbies': [],
        'Languages': [],
        'LinkedIn': '',
        'GitHub': ''
    }
    
    # Ensure text is a string to avoid issues with NaN
    if not isinstance(text, str):
        return sections  # Return empty sections if the text is invalid
    
    # Split text into paragraphs
    paragraphs = text.split('\n')

    # Define keywords for sections (case-insensitive)
    name_keywords = ['Name']
    contact_keywords = ['Phone', 'Email', 'Contact']
    summary_keywords = ['PROFESSIONAL SUMMARY', 'Summary']
    skills_keywords = ['SKILLS', 'Technical Skills']
    education_keywords = ['EDUCATION', 'Education', 'Degrees']
    experience_keywords = ['PROFESSIONAL EXPERIENCE', 'EXPERIENCE']
    certifications_keywords = ['CERTIFICATES', 'CERTIFICATIONS']
    accomplishments_keywords = ['ACCOMPLISHMENTS', 'Achievements']
    hobbies_keywords = ['HOBBIES', 'Interests']
    languages_keywords = ['LANGUAGES', 'Language']
    linkedin_keywords = ['LinkedIn', 'linkedin.com']
    github_keywords = ['GitHub', 'github.com']

    current_section = None
    
    for paragraph in paragraphs:
        paragraph = paragraph.strip()  # Clean up whitespace
        
        # Skip empty lines
        if not paragraph:
            continue
        
        # Extract LinkedIn and GitHub links directly
        if any(kw.lower() in paragraph.lower() for kw in linkedin_keywords):
            sections['LinkedIn'] = paragraph
            continue
        if any(kw.lower() in paragraph.lower() for kw in github_keywords):
            sections['GitHub'] = paragraph
            continue

        # Extract Name and Contact information
        if any(kw.lower() in paragraph.lower() for kw in name_keywords) or re.match(r'\b[A-Za-z]+\s[A-Za-z]+', paragraph):
            sections['Name'] = paragraph
            continue
        if any(kw.lower() in paragraph.lower() for kw in contact_keywords) or re.search(r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b', paragraph):
            sections['Contact'] = paragraph
            continue
        
        # Check if paragraph starts a new section
        if any(kw.lower() in paragraph.lower() for kw in summary_keywords):
            current_section = 'Professional Summary'
            continue
        elif any(kw.lower() in paragraph.lower() for kw in skills_keywords):
            current_section = 'Skills'
            continue
        elif any(kw.lower() in paragraph.lower() for kw in education_keywords):
            current_section = 'Education'
            continue
        elif any(kw.lower() in paragraph.lower() for kw in experience_keywords):
            current_section = 'Experience'
            continue
        elif any(kw.lower() in paragraph.lower() for kw in certifications_keywords):
            current_section = 'Certifications'
            continue
        elif any(kw.lower() in paragraph.lower() for kw in accomplishments_keywords):
            current_section = 'Accomplishments'
            continue
        elif any(kw.lower() in paragraph.lower() for kw in hobbies_keywords):
            current_section = 'Hobbies'
            continue
        elif any(kw.lower() in paragraph.lower() for kw in languages_keywords):
            current_section = 'Languages'
            continue

        # Extract Professional Summary from the first few lines (before any section headers)
        if current_section == 'Professional Summary' and len(paragraph) > 0:
            sections['Professional Summary'] += paragraph + ' '
        
        # Add content to the current section
        if current_section == 'Skills':
            sections['Skills'].append(paragraph)
        elif current_section == 'Education':
            sections['Education'].append(paragraph)
        elif current_section == 'Experience':
            sections['Experience'].append(paragraph)
        elif current_section == 'Certifications':
            sections['Certifications'].append(paragraph)
        elif current_section == 'Accomplishments':
            sections['Accomplishments'].append(paragraph)
        elif current_section == 'Hobbies':
            sections['Hobbies'].append(paragraph)
        elif current_section == 'Languages':
            sections['Languages'].append(paragraph)
    
    # Ensure 'N/A' if any section is empty
    for key in sections.keys():
        if key in ['Name', 'Contact', 'LinkedIn', 'GitHub', 'Professional Summary'] and not sections[key]:
            sections[key] = 'N/A'
        elif not sections[key]:
            sections[key] = ['N/A']

    return sections

# Read CSV file
df = pd.read_csv('resumes_data.csv')  # Replace with your CSV file path

# Handle NaN values in the 'Resume' column
# df['Resume'].fillna('', inplace=True) # This is deprecated
df['Resume'] = df['Resume'].fillna('')

resume_data = df['Resume'].tolist()

# Process each resume and extract sections
extracted_data = []
for text in resume_data:
    extracted_data.append(extract_sections(text))

# Create a DataFrame to store structured data
structured_df = pd.DataFrame(extracted_data)

# Save the structured data to a new CSV file
structured_df.to_csv('structured_resumes.csv', index=False)
print("Structured data has been extracted and saved to 'structured_resumes.csv'.")
